# Question Answering with LangChain, OpenAI, and MultiQuery Retriever

This interactive workbook demonstrates example of Elasticsearch's [MultiQuery Retriever](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) to generate similar queries for a given user input and apply all queries to retrieve a larger set of relevant documents from a vectorstore.

Before we begin, we first split the fictional workplace documents into passages with `langchain` and uses OpenAI to transform these passages into embeddings and then store these into Elasticsearch.

We will then ask a question, generate similar questions using langchain and OpenAI, retrieve relevant passages from the vector store, and use langchain and OpenAI again to provide a summary for the questions.

## Install packages and import modules

In [69]:
#!pip install langchain-elasticsearch

In [70]:
#!python3 -m pip install -qU jq lark langchain langchain-elasticsearch langchain_openai tiktoken


from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai.llms import OpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from getpass import getpass

## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial. 

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment.

We will use [ElasticsearchStore](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html) to connect to our elastic cloud deployment, This would help create and index data easily.  We would also send list of documents that we created in the previous step

In [71]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
ELASTIC_CLOUD_ID = os.getenv('ELASTIC_CLOUD_ID')
ELASTIC_API_KEY = os.getenv('ELASTIC_API_KEY')

In [72]:
""" # https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# https://platform.openai.com/api-keys
OPENAI_API_KEY = getpass("OpenAI API key: ") """

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name='chatbot_index', #give it a meaningful name,
    embedding=embeddings,
)

## Indexing Data into Elasticsearch
Let's download the sample dataset and deserialize the document.

In [73]:
from urllib.request import urlopen
import json

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/chatbot-rag-app/data/data.json"

response = urlopen(url)
data = json.load(response)

with open("temp.json", "w") as json_file:
    json.dump(data, json_file)

### Split Documents into Passages

We’ll chunk documents into passages in order to improve the retrieval specificity and to ensure that we can provide multiple passages within the context window of the final question answering prompt.

Here we are chunking documents into 800 token passages with an overlap of 400 tokens.

Here we are using a simple splitter but Langchain offers more advanced splitters to reduce the chance of context being lost.

In [74]:
#!pip install jq

In [75]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

#Populate the metadata dictionary with keys name, summary, url, category, and updated_at.
def metadata_func(record: dict, metadata: dict) -> dict:
    # Check for required fields and populate metadata
    if all(field in record for field in ["name", "summary", "url", "category", "updated_at"]):
        metadata["name"] = record.get("name")
        metadata["summary"] = record.get("summary")
        metadata["url"] = record.get("url")
        metadata["category"] = record.get("category")
        metadata["updated_at"] = record.get("updated_at")
        return metadata
    else:
        # If any required field is missing, return None
        print(f"Skipping document with missing metadata: {record}")
        return None


# For more loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/
# And 3rd party loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/#third-party-loaders

loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".[]",
    content_key="content",
    metadata_func=metadata_func,
)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=800, chunk_overlap=400 #define chunk size and chunk overlap
)
docs = loader.load_and_split(text_splitter=text_splitter)

' text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(\n    chunk_size=800, chunk_overlap=400 #define chunk size and chunk overlap\n)\ndocs = loader.load_and_split(text_splitter=text_splitter) '

In [57]:
for doc in docs:
    print(doc.metadata)  # Verify the metadata includes 'name'

{'source': 'C:\\Users\\KK\\Documents\\$$$IRONHACK\\7-Week-LangChain-RAG-Agents\\Day34\\lab-chatbot-with-multi-query-retriever\\temp.json', 'seq_num': 1, 'name': 'Work From Home Policy', 'summary': 'This policy outlines the guidelines for full-time remote work, including eligibility, equipment and resources, workspace requirements, communication expectations, performance expectations, time tracking and overtime, confidentiality and data security, health and well-being, and policy reviews and updates. Employees are encouraged to direct any questions or concerns', 'url': './sharepoint/Work from home policy.txt', 'category': 'teams', 'updated_at': '2020-03-01'}
{'source': 'C:\\Users\\KK\\Documents\\$$$IRONHACK\\7-Week-LangChain-RAG-Agents\\Day34\\lab-chatbot-with-multi-query-retriever\\temp.json', 'seq_num': 2, 'name': 'April Work From Home Update', 'summary': 'Starting May 2022, employees will need to work two days a week in the office. Coordinate with your supervisor and HR department fo

### Bulk Import Passages

Now that we have split each document into the chunk size of 800, we will now index data to elasticsearch using [ElasticsearchStore.from_documents](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html#langchain.vectorstores.elasticsearch.ElasticsearchStore.from_documents).

We will use Cloud ID, Password and Index name values set in the `Create cloud deployment` step.

In [58]:
# Continue with the retriever process using split_docs (which are valid)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="chatbot_index",
    embedding=embeddings,
)

retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)

print(f"{len(split_docs)} valid documents processed and indexed.")


15 valid documents processed and indexed.


# Question Answering with MultiQuery Retriever

Now that we have the passages stored in Elasticsearch, we can now ask a question to get the relevant passages.

In [63]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document

import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as verbose and educational in your response as possible. 
    
    context: {context}
    Question: "{question}"
    Answer:
    """
)

LLM_DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
SOURCE: {name}
{page_content}
---
"""
)

' def _combine_documents(\n    docs, document_prompt=LLM_DOCUMENT_PROMPT, document_separator="\n\n"\n):\n    doc_strings = [format_document(doc, document_prompt) for doc in docs]\n    return document_separator.join(doc_strings)\n\n\n_context = RunnableParallel(\n    context=retriever | _combine_documents,\n    question=RunnablePassthrough(),\n)\n\nchain = _context | LLM_CONTEXT_PROMPT | llm\n\nans = chain.invoke("what is the nasa sales team?")\n\nprint("---- Answer ----")\nprint(ans) '

In [64]:
def _combine_documents(docs, document_prompt=LLM_DOCUMENT_PROMPT, document_separator="\n\n"):
    doc_strings = []
    
    for doc in docs:
        # Check if both 'name' and 'page_content' are present
        if 'name' not in doc.metadata:
            print(f"Missing 'name' in document metadata: {doc.metadata}")
            continue  # Skip this document
        
        if not doc.page_content:
            print(f"Missing 'page_content' in document: {doc.metadata}")
            continue  # Skip this document
        
        # If both are present, format the document
        doc_strings.append(format_document(doc, document_prompt))
    
    # Join all the formatted document strings
    return document_separator.join(doc_strings)


In [65]:
_context = RunnableParallel(
    context=retriever | _combine_documents,
    question=RunnablePassthrough(),
)

# Run the chain
chain = _context | LLM_CONTEXT_PROMPT | llm

ans = chain.invoke("how to ensure correct tax deduction?")
print("---- Answer ----")
print(ans)
 

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are some methods for guaranteeing accurate tax deductions?', '2. How can I make sure that my tax deductions are correct?', '3. What steps should I take to ensure the accuracy of my tax deductions?']


Missing 'name' in document metadata: {'source': 'C:\\Users\\KK\\Documents\\$$$IRONHACK\\7-Week-LangChain-RAG-Agents\\Day34\\lab-chatbot-with-multi-query-retriever\\temp.json', 'seq_num': 14}
Missing 'name' in document metadata: {'source': 'C:\\Users\\KK\\Documents\\$$$IRONHACK\\7-Week-LangChain-RAG-Agents\\Day34\\lab-chatbot-with-multi-query-retriever\\temp.json', 'seq_num': 15}
---- Answer ----
To ensure correct tax deductions, it is important to update your tax elections forms, specifically the TD1 Personal Tax Credits Return form. This form can be accessed through the Canada Revenue Agency (CRA) website and should be filled out with accurate personal information and any applicable tax credits. It is important to carefully read the instructions for each section to ensure the correct amounts are claimed. Once completed, the form should be signed and submitted to your employer, who will use the information to calculate the correct amount of tax to be deducted from your pay. It is also 

**Generate at least two new iteratioins of the previous cells - Be creative.** Did you master Multi-
Query Retriever concepts through this lab?